# UVIT view of Centaurus A; a detailed study on positive AGN feedback
## Jupyter notebooks used in the study: Notebook 2

We are making the scripts used for the study publicly available. This notebook is associated with the section 2.2 of the article. 

The cell below enable jupyter widget for matplotlib. It can produce interactive matplotlib inline plots. Please comment it if you do not have it installed. 

In [1]:
%matplotlib widget

Please install the packages if you do not have them already. 

In [2]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.wcs import WCS
from astropy.stats import gaussian_fwhm_to_sigma, SigmaClip
from astropy.table import join, Table, setdiff
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord
from astropy.convolution import Gaussian2DKernel, convolve
from astroquery.xmatch import XMatch
from photutils.aperture import CircularAperture
from photutils.detection import DAOStarFinder
from photutils.background import Background2D, SExtractorBackground
from photutils.segmentation import detect_threshold, detect_sources, deblend_sources, SourceCatalog

Could not import regions, which is required for some of the functionalities of this module.


In [3]:
position = SkyCoord('13:26:16.9588 -42:51:36.902', frame='icrs', unit=(u.hourangle, u.deg))
size = (16.99948 * u.arcmin, 14.07162 * u.arcmin)

# distance to centaurus A. 
distance = 3.8 #mpc
pixel_scale = 0.416858 #arcseconds
pixelscale_in_Kpc = distance * 1e3 * np.deg2rad(pixel_scale / 3600)
pixelarea_in_kpc = pixelscale_in_Kpc ** 2

In [4]:
where_is_data = 'data/'
F148W_hdu = fits.open(where_is_data + 'wcs_AS1G08_023T01_9000001978uvtFIIPC00F1A_l2imb.fits.gz')
F148W_error_hdu = fits.open(where_is_data + 'wcs_AS1G08_023T01_9000001978uvtFIIPC00F1A_l2erb.fits.gz')

final_tbl = Table.read('final_table.fits') # generated by Notebook1

F148W_wcs = WCS(F148W_hdu[0].header)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [5]:
F148W_pixel_scale = (F148W_wcs.proj_plane_pixel_scales()[1].to(u.arcsec) + F148W_wcs.proj_plane_pixel_scales()[0].to(u.arcsec)) / 2
F148W_pixel_scale = F148W_pixel_scale.value

In [6]:
F148W_cutout = Cutout2D(F148W_hdu[0].data, position, size, F148W_wcs)
F148W_error_cutout = Cutout2D(F148W_error_hdu[0].data, position, size, F148W_wcs)

In [7]:
sigma_clip = SigmaClip(sigma = 3., maxiters = 10)
bkg_estimator = SExtractorBackground()

F148W_bkg = Background2D(F148W_cutout.data, (400, 400), filter_size = (3, 3), sigma_clip = sigma_clip,
                       bkg_estimator = bkg_estimator)

In [8]:
F148W_fwhm = (1.4 / F148W_pixel_scale)  # FUV FWHM = 1.4 arcseconds
F148W_sigma = (1.4 / F148W_pixel_scale) * gaussian_fwhm_to_sigma  # FUV FWHM = 1.4 arcseconds
F148W_kernel = Gaussian2DKernel(F148W_sigma)
F148W_smoothed_data = convolve(F148W_cutout.data, F148W_kernel)
F148W_smoothed_bkg = convolve(F148W_bkg.background, F148W_kernel)

daofind = DAOStarFinder(fwhm = F148W_fwhm, threshold = 1 * np.median(F148W_bkg.background_rms))  
sources = daofind(F148W_smoothed_data - F148W_smoothed_bkg)  

In [9]:
len(sources)

146

In [10]:
skycoo = F148W_cutout.wcs.pixel_to_world(sources['xcentroid'], sources['ycentroid'])
sources['RA_J2000'] = skycoo.ra.degree
sources['DEC_J2000'] = skycoo.dec.degree

In [11]:
GAIA_stars_tbl = XMatch.query(cat1 = sources,
                              cat2 = 'vizier:I/352/gedr3dis',
                              max_distance = 1 * u.arcsec, 
                              colRA1 = 'RA_J2000',
                              colDec1 = 'DEC_J2000')

not_stars_tbl = setdiff(sources, GAIA_stars_tbl, keys = 'id')

In [12]:
len(not_stars_tbl)

102

In [13]:
not_stars_tbl_coo = SkyCoord(ra = not_stars_tbl['RA_J2000'] * u.degree, dec = not_stars_tbl['DEC_J2000'] * u.degree)
final_tbl_coo = SkyCoord(ra = final_tbl['RA_J2000'] * u.degree, dec = final_tbl['DEC_J2000'] * u.degree)

In [14]:
# Star-forming sources
max_sep = 2.0 * u.arcsec
idx, d2d, d3d = final_tbl_coo.match_to_catalog_sky(not_stars_tbl_coo)
sep_constraint = d2d < max_sep
SF_sources_tbl = final_tbl[sep_constraint]
SF_sources_tbl.write('SF_sources.fits', format = 'fits', overwrite = True)

In [15]:
len(SF_sources_tbl)

83

In [16]:
# diffuse sources
nonSF_sources_tbl = final_tbl[~sep_constraint]
nonSF_sources_tbl.write('nonSF_sources.fits', format = 'fits', overwrite = True)

In [17]:
len(nonSF_sources_tbl)

269